In [33]:
from collections import OrderedDict
import json
import pprint

In [4]:
input_str= """
{
    "manual": [
        "AG(stab -> EF(P5))",
        "AG( ((CS_AL1_W < 4) | (CS_AL1_W > 8)) -> AF(!(P5 | P6)) )"
    ]            
}
"""

In [5]:
in_dict= json.loads(input_str, object_pairs_hook= OrderedDict)

In [17]:
# Expected NuSMV
expected= "SPEC\n" + \
    "AG(stab -> EF(P5));\n" + \
    "SPEC\n" + \
    "AG( ((CS_AL1_W < 4) | (CS_AL1_W > 8)) -> AF(!(P5 | P6)) );\n"

In [18]:
expected

'SPEC\nAG(stab -> EF(P5));\nSPEC\nAG( ((CS_AL1_W < 4) | (CS_AL1_W > 8)) -> AF(!(P5 | P6)) );\n'

In [11]:
# Output string
out_str= ""

In [12]:
# Loop through items
for spec in in_dict["manual"]:
    out_str= out_str + "SPEC\n" + spec + ";\n"

In [16]:
out_str

'SPEC\nAG(stab -> EF(P5));\nSPEC\nAG( ((CS_AL1_W < 4) | (CS_AL1_W > 8)) -> AF(!(P5 | P6)) );\n'

In [19]:
assert(out_str == expected)

## Auto Specification

In [21]:
# Json Input
input_str= r"""
{
    "places": {
        "P1": [
            [
                ["OUT1"], ["OUT2"]
            ],
            "Initial place"
        ],
        "P2": [
            [
                ["OUT1"], ["OUT2"]
            ],
            ["X_SET:= 560"],
            "Move tool to X position"
        ],
        "STOP": [
            [
                [], ["OUT1", "OUT2"]
            ],
            ["X_SET:= 0"],
            "Stop machining"
        ],
        "initial": ["P1"]
    },
    "outputs": {
        "OUT1": ["%QX100.0"],
        "OUT2": ["%QX100.1"],
        "X_SET": ["%QW100"]
    },
    "transitions": {
        "T1": [
            ["P1"],
            [
                [
                    ["IN1"], ["IN2"]
                ]
            ],
            ["P2"]
        ],
        "T2": [
            ["P2"],
            [
                [
                    "X_POS= 560"
                ]
            ],
            ["STOP"]
        ],
        "T3": [
            ["STOP"],
            [
                [
                    ["START"], []
                ]
            ],
            ["P1"]
        ]
    },
    "inputs": {
        "IN1": [
            "boolean", "%IX100.0"
        ],
        "IN2": [
            "boolean", "%IX100.1"
        ],
        "X_POS": [
            [
                "{0, 560}",
                0,
                "{0, 560}"
            ],
            "%IW100"
        ]
    },
    "specifications": {
        "auto": true,
        "manual": [
            "AG(EF(STOP))"
        ]
    }
}
"""

In [22]:
# Expected NuSMV
expected= "--LIVENESS\n" +\
    "SPEC\n" +\
    "AG(EF(T1));\n" +\
    "SPEC\n" +\
    "AG(EF(T2));\n" +\
    "SPEC\n" +\
    "AG(EF(T3));\n" +\
    "--DEADLOCK-FREENESS\n" +\
    "SPEC\n" +\
    "AG(EF( T1 | T2 | T3 ));\n" +\
    "--NO DEAD TRANSITION\n" +\
    "SPEC\n" +\
    "EF(T1);\n" +\
    "SPEC\n" +\
    "EF(T2);\n" +\
    "SPEC\n" +\
    "EF(T3);\n" +\
    "--NO UNSTABLE CYCLE\n" +\
    "SPEC\n" +\
    "AG(EF(stab));\n" +\
    "--NO CONTRADICTORY OUTPUTS\n" +\
    "SPEC\n" +\
    "AG( ! (set_OUT1 & reset_OUT1) );\n" +\
    "SPEC\n" +\
    "AG( ! (set_OUT2 & reset_OUT2) );\n" +\
    "-- NO EMPTY OUTPUT\n" +\
    "SPEC\n" +\
    "AG( set_OUT1 | reset_OUT1 );\n" +\
    "SPEC\n" +\
    "AG( set_OUT2 | reset_OUT2 );\n"

In [23]:
# Convert json to ordered dict
in_dict= json.loads(input_str, object_pairs_hook=OrderedDict)

In [24]:
in_dict

OrderedDict([('places',
              OrderedDict([('P1', [[['OUT1'], ['OUT2']], 'Initial place']),
                           ('P2',
                            [[['OUT1'], ['OUT2']],
                             ['X_SET:= 560'],
                             'Move tool to X position']),
                           ('STOP',
                            [[[], ['OUT1', 'OUT2']],
                             ['X_SET:= 0'],
                             'Stop machining']),
                           ('initial', ['P1'])])),
             ('outputs',
              OrderedDict([('OUT1', ['%QX100.0']),
                           ('OUT2', ['%QX100.1']),
                           ('X_SET', ['%QW100'])])),
             ('transitions',
              OrderedDict([('T1', [['P1'], [[['IN1'], ['IN2']]], ['P2']]),
                           ('T2', [['P2'], [['X_POS= 560']], ['STOP']]),
                           ('T3', [['STOP'], [[['START'], []]], ['P1']])])),
             ('inputs',
              Ordere

#### Check if auto specs are requested

In [25]:
try:
    if in_dict["specifications"]["auto"]:
        gen_auto= True
    else:
        gen_auto= False
except KeyError:
    gen_auto= False

In [26]:
gen_auto

True

#### Liveness, deadlock-freeness, and dead transitions

In [27]:
# Transitions
transitions= list(in_dict["transitions"])

In [28]:
transitions

['T1', 'T2', 'T3']

In [30]:
liveness= "--CHECK LIVENESS\n"
deadlock_st= "--CHECK DEADLOCKS\n" + "SPEC\n" + "AG(EF("
deadlock= ""
deadlock_end= " ));\n"
dead= "--CHECK DEAD TRANSITIONS\n"

In [31]:

for idx,transition in enumerate(transitions):
    liveness= liveness +\
        "SPEC\n" +\
        "AG(EF({0}));\n".format(transition)

    dead= dead +\
        "SPEC\n" +\
        "EF({0});\n".format(transition)

    # Check if last element
    if (idx+1) == len(transitions):
        deadlock= deadlock + " {0}".format(transition)
    # Otherwise
    else:
        deadlock= deadlock + " {0} |".format(transition)



In [35]:
liveness

'--CHECK LIVENESS\nSPEC\nAG(EF(T1));\nSPEC\nAG(EF(T2));\nSPEC\nAG(EF(T3));\n'

In [36]:
deadlock= deadlock_st + deadlock + deadlock_end

In [37]:
deadlock

'--CHECK DEADLOCKS\nSPEC\nAG(EF(  T1 | T2 | T3 ));\n'

In [38]:
dead

'--CHECK DEAD TRANSITIONS\nSPEC\nEF(T1);\nSPEC\nEF(T2);\nSPEC\nEF(T3);\n'

#### Unstable cycles

In [40]:
unstable_cycles= "--CHECK UNSTABLE CYCLES\n" + "SPEC\n" + "AG(EF(stab));\n"

#### Contradictory and empty outputs

In [49]:
bool_outputs= [key for key in list(in_dict["outputs"]) if "%QX" in in_dict["outputs"][key][0] ]

In [50]:
bool_outputs

['OUT1', 'OUT2']

In [51]:
contradictory= "--CHECK CONTRADICTORY OUTPUTS\n" 
empty= "--CHECK EMPTY OUTPUTS\n"

In [52]:
for output in bool_outputs:
    contradictory= contradictory +\
        "SPEC\n" +\
        "AG( ! (set_{0} & reset_{0}) );\n".format(output)

    empty= empty +\
        "SPEC\n" +\
        "AG( set_{0} | reset_{0} );\n".format(output)

In [53]:
contradictory

'SPEC\nAG( ! (set_OUT1 & reset_OUT1) );\nSPEC\nAG( ! (set_OUT2 & reset_OUT2) );\n'

In [54]:
empty

'SPEC\nAG( set_OUT1 | reset_OUT1 );\nSPEC\nAG( set_OUT2 | reset_OUT2 );\n'